# Basics

### Imports and Data Loarding

In [1]:
import os
import pandas as pd
import numpy as np
from untrade.client import Client
import plotly.graph_objects as go

# Load ETH data
eth_data = pd.read_csv("ETH/ETHUSDT_15m.csv")

print("Ethereum data:")
print(eth_data.head(), "\n", "Shape:", eth_data.shape)

# Load BTC data
btc_data = pd.read_csv("BTC/BTC_2019_2023_15m.csv")

print("\n\n Bitcoin data:")
print(btc_data.head(), "\n", "Shape:", btc_data.shape)



Ethereum data:
   Unnamed: 0             datetime    open    high     low   close    volume
0           0  2019-12-01 00:00:00  151.38  151.38  150.09  150.41  4472.159
1           1  2019-12-01 00:15:00  150.50  150.74  150.16  150.30  3656.530
2           2  2019-12-01 00:30:00  150.30  150.83  150.23  150.48  4620.579
3           3  2019-12-01 00:45:00  150.46  150.61  150.18  150.60  4178.315
4           4  2019-12-01 01:00:00  150.61  150.64  148.60  148.78  5234.745 
 Shape: (143233, 7)


 Bitcoin data:
   Unnamed: 0             datetime     open     high      low    close  volume
0           0  2019-09-08 17:45:00  10000.0  10000.0  10000.0  10000.0   0.002
1           1  2019-09-08 18:00:00  10000.0  10000.0  10000.0  10000.0   0.000
2           2  2019-09-08 18:15:00  10000.0  10000.0  10000.0  10000.0   0.000
3           3  2019-09-08 18:30:00  10000.0  10000.0  10000.0  10000.0   0.000
4           4  2019-09-08 18:45:00  10000.0  10000.0  10000.0  10000.0   0.000 
 Shape: (1

### Basic Functions


In [2]:
def strat(data):
    """
    Apply strategy to filter consecutive signals.

    Parameters:
    - data (DataFrame): Dataframe containing buy/sell signals.

    Returns:
    - data (DataFrame): Dataframe with filtered signals.
    """

    signal = []
    prev = None
    for value in data["Signal"]:
        if value == prev:
            signal.append(0)
        else:
            signal.append(value)
        prev = value

    data["signals"] = signal

    # Keep only the required columns
    data = data[['datetime', 'open', 'high', 'low',
                'close', 'volume', 'signals', 'trade_type', 'SL', 'TP']]
    
    return data


def perform_backtest(csv_file_path):
    """
    Perform backtesting using the untrade SDK.

    Parameters:
    - csv_file_path (str): Path to the CSV file containing historical price data and signals.

    Returns:
    - result (generator): Generator object that yields backtest results.
    """

    # Create an instance of the untrade client
    client = Client()

    # Perform backtest using the provided CSV file path
    result = client.backtest(
        jupyter_id="vraj2811",  # your Jupyter ID
        file_path=csv_file_path,
        leverage=1,  # Adjust leverage as needed
    )

    return result

### Plotting Functions

In [3]:
# Plotting the candles

def plot_candles(df, ema_span=50):
    """
    Visualizes the candles on an interactive candlestick chart with EMA50.
    
    Parameters:
        df (pd.DataFrame): DataFrame with columns ['datetime', 'open', 'high', 'low', 'close'].
    """
    
    col_name = 'EMA'+str(ema_span)
    df[col_name] = df['close'].ewm(span=ema_span, adjust=False).mean()

    fig = go.Figure()

    # Add Candlestick Chart
    fig.add_trace(go.Candlestick(
        x=df['datetime'],
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        increasing_line_color='green',
        decreasing_line_color='red',
        name="Candles"
    ))

    # Add EMA50 Line
    fig.add_trace(go.Scatter(
        x=df['datetime'],
        y=df[col_name],
        mode='lines',
        line=dict(color='blue', width=2),
        name="EMA"+str(ema_span)
    ))

    # Layout Settings
    fig.update_layout(
        title="Candlestick Chart",
        xaxis_title="Datetime",
        yaxis_title="Price",
        xaxis_rangeslider_visible=False
    )

    fig.show()


# Heiken-Ashi

### Conversion Functions

In [4]:
def convert_to_heiken_ashi(data):
    """
    Convert the input data to Heiken Ashi candlesticks.

    Parameters:
    - data (DataFrame): Input data with columns ['datetime', 'open', 'high', 'low', 'close', 'volume'].

    Returns:
    - ha_data (DataFrame): Dataframe with Heiken Ashi candles.
    """

    ha_data = data.copy()
    ha_data['ha_close'] = (data['open'] + data['high'] + data['low'] + data['close']) / 4
    ha_data['ha_open'] = (data['open'].shift(1) + data['close'].shift(1)) / 2
    ha_data['ha_high'] = ha_data[['ha_open', 'ha_close', 'high']].max(axis=1)
    ha_data['ha_low'] = ha_data[['ha_open', 'ha_close', 'low']].min(axis=1)
    
    # drop original high, low, open, close columns
    ha_data.drop(['open', 'high', 'low', 'close'], axis=1, inplace=True)
    ha_data = ha_data.rename(columns={'ha_open': 'open', 'ha_high': 'high', 'ha_low': 'low', 'ha_close': 'close'})
    
    return ha_data



In [5]:
chopped_eth = eth_data.iloc[0:150].copy()
ha_eth = convert_to_heiken_ashi(chopped_eth.copy())

plot_candles(chopped_eth)
plot_candles(ha_eth)


In [6]:
chopped_btc = btc_data.iloc[0:150].copy()
ha_btc = convert_to_heiken_ashi(chopped_btc.copy())

plot_candles(chopped_btc)
plot_candles(ha_btc)

### Strategy Functions

## Long Position:

### Entry

- 2 consecutive green candles + current price > 50 EMA
- Entry_Price = Close of the current candle

### Stop_Loss 

- Stop_Loss = low - (high - low) * SLF
- SLF = Stop Loss Factor (Ideally 5-10%)

- Can add Dynamic Stop Loss to gain No Loss or Profit Stop Loss. 
    - While in long position, update the Stop Loss according to the formula:
    - Stop_Loss = max(Stop_Loss, low - (high - low) * SLF)

### Take_Profit

- Take_Profit = Entry_Price + (Entry_Price - Stop_Loss) * RRR
- RRR = Risk Reward Ratio (Ideally 1:2 or 1:3)

### Exit

- Exit when we find a doji or a red candle above the 50 EMA

## Short Position:

### Entry

- 2 consecutive red candles + current price < 50 EMA
- Entry_Price = Close of the current candle

### Stop_Loss

- Stop_Loss = high + (high - low) * SLF
- SLF = Stop Loss Factor (Ideally 5-10%)

- Can add Dynamic Stop Loss to gain No Loss or Profit Stop Loss.
    - While in short position, update the Stop Loss according to the formula:
    - Stop_Loss = min(Stop_Loss, high + (high - low) * SLF)

### Take_Profit

- Take_Profit = Entry_Price - (Stop_Loss - Entry_Price) * RRR
- RRR = Risk Reward Ratio (Ideally 1:2 or 1:3)

### Exit

- Exit when we find a doji or a green candle below the 50 EMA


In [7]:
def heiken_ashi_strategy(df, SLF=0.05, RRR=2, ema_span=50):
    """
    Apply Heiken Ashi strategy to the input data.

    Parameters:
    - data (DataFrame): Input data with columns ['datetime', 'open', 'high', 'low', 'close', 'volume'].

    Returns:
    - data (DataFrame): Dataframe with buy/sell Signal.
    """

    # Calculate the 50 EMA
    df['EMA_50'] = df['close'].ewm(span=ema_span, adjust=False).mean()

    ha_df = convert_to_heiken_ashi(df)

    # Add columns for Signal and trade_type
    df['Signal'] = 0
    df['trade_type'] = ''
    df['SL'] = 0
    df['TP'] = 0
    
    # Variables to track open positions
    in_long_position = False
    in_short_position = False
    stop_loss = None
    entry_price = None
    active_trade = False

    # Loop through the rows of the dataframe
    for i in range(1, len(df)):
        # Identify the current and previous candles
        current_candle = ha_df.iloc[i]
        prev_candle = ha_df.iloc[i-1]
        current_true_candle = df.iloc[i-1]


        if active_trade:

            # Default to HOLD
            df.at[i, 'Signal'] = 0
            df.at[i, 'trade_type'] = 'HOLD'

            # Long Position Dynamic Stop Loss Update
            if in_long_position and current_true_candle['close'] > entry_price:
                stop_loss = max(stop_loss, current_true_candle['low'] - (current_true_candle['high'] - current_true_candle['low']) * SLF)
                df.at[i, 'SL'] = stop_loss
                df.at[i, 'TP'] = take_profit

            # If limits are hit, close the position
            if in_long_position and (current_true_candle['close'] >= take_profit or current_true_candle['close'] <= stop_loss):
                df.at[i, 'Signal'] = -1 # Sell Signal
                df.at[i, 'trade_type'] = 'CLOSE'
                in_long_position = False
                active_trade = False

            # Exit condition for Long: Doji or Red candle
            if in_long_position:
                if current_candle['close'] < current_candle['open']:  
                    df.at[i, 'Signal'] = -1  
                    df.at[i, 'trade_type'] = 'CLOSE'
                    in_long_position = False
                    active_trade = False

            # Short Position Dynamic Stop Loss Update
            if in_short_position and current_true_candle['close'] < entry_price:
                stop_loss = min(stop_loss, current_true_candle['high'] + (current_true_candle['high'] - current_true_candle['low']) * SLF)
                df.at[i, 'SL'] = stop_loss
                df.at[i, 'TP'] = take_profit
                df.at[i, 'Signal'] = 0
                df.at[i, 'trade_type'] = 'HOLD'

            # If limits are hit, close the position
            if in_short_position and (current_true_candle['close'] <= take_profit or current_true_candle['close'] >= stop_loss):
                df.at[i, 'Signal'] = 1 # Buy Signal
                df.at[i, 'trade_type'] = 'CLOSE'
                in_short_position = False
                active_trade = False

            # Exit condition for Short: Green candle or Doji
            if in_short_position:
                if current_candle['close'] > current_candle['open']:  
                    df.at[i, 'Signal'] = 1 # Buy Signal
                    df.at[i, 'trade_type'] = 'CLOSE'
                    in_short_position = False
                    active_trade = False

        else:
            # Long Position Conditions
            # Entry condition: 2 consecutive green candles + price > 50 EMA
            if (current_candle['close'] > current_candle['open'] and current_candle['low']==current_candle['open']) and prev_candle['close'] > prev_candle['open'] and current_candle['close'] > current_candle['EMA_50']:
                entry_price = current_true_candle['close']
                stop_loss = current_true_candle['low'] - (current_true_candle['high'] - current_true_candle['low']) * SLF
                take_profit = entry_price + (entry_price - stop_loss) * RRR
                df.at[i, 'SL'] = stop_loss
                df.at[i, 'TP'] = take_profit
                df.at[i, 'Signal'] = 1  # Buy Signal
                df.at[i, 'trade_type'] = 'LONG'
                in_long_position = True
                active_trade = True

            # Short Position Conditions
            # Entry condition: 2 consecutive red candles + price < 50 EMA
            if (current_candle['close'] < current_candle['open'] and current_candle['open']==current_candle['high']) and prev_candle['close'] < prev_candle['open'] and current_candle['close'] < current_candle['EMA_50']:
                entry_price = current_true_candle['close']
                stop_loss = current_true_candle['high'] + (current_true_candle['high'] - current_true_candle['low']) * SLF
                take_profit = entry_price - (stop_loss - entry_price) * RRR
                df.at[i, 'SL'] = stop_loss
                df.at[i, 'TP'] = take_profit
                df.at[i, 'Signal'] = -1  # Sell Signal
                df.at[i, 'trade_type'] = 'SHORT'
                in_short_position = True
                active_trade = True
        
        df['SL'] = df['SL'].astype(float)
        df['TP'] = df['TP'].astype(float)
        active_trade = in_long_position or in_short_position


    # Return the updated dataframe
    return df

In [8]:

eth_data = heiken_ashi_strategy(eth_data, 0.05, 3, 50)
eth_data = strat(eth_data)

os.makedirs('ETH/processed', exist_ok=True)
# Make a directory under ETH folder with the name 'processed' and save the processed data in it
eth_data.to_csv('ETH/processed/ETH_15m.csv', index=False)

eth_csv_path = 'ETH/processed/ETH_15m.csv'
eth_backtest = perform_backtest(eth_csv_path)

for value in eth_backtest:
    print(value)


data: {
  "jupyter_id": "vraj2811",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-11-29 00:00:00",
      "Total Trades": 29059,
      "Leverage Applied": 1.0,
      "Winning Trades": 4901,
      "Losing Trades": 24158,
      "No. of Long Trades": 14514,
      "No. of Short Trades": 14545,
      "Benchmark Return(%)": 1467.125199,
      "Benchmark Return(on $1000)": 14671.251994,
      "Win Rate": 16.865687,
      "Winning Streak": 4,
      "Losing Streak": 80,
      "Gross Profit": -1118.989806,
      "Net Profit": -44707.489806,
      "Average Profit": -1.538508,
      "Maximum Drawdown(%)": 4470.748981,
      "Average Drawdown(%)": 2277.80172,
      "Largest Win": 118.912225,
      "Average Win": 6.840802,
      "Largest Loss": -93.257304,
      "Average Loss": -3.238441,
      "Maximum Holding Time": "0 days 11:44:59",
      "Average Holding Time": "0 days 0:26:10",
      "Maximum Adverse Excursion": 10.69524

In [10]:
btc_data = heiken_ashi_strategy(btc_data, 0.05, 3, 50)
btc_data = strat(btc_data)

os.makedirs('BTC/processed', exist_ok=True)
# Make a directory under ETH folder with the name 'processed' and save the processed data in it
btc_data.to_csv('BTC/processed/BTC_15m.csv', index=False)

btc_csv_path = 'BTC/processed/BTC_15m.csv'
btc_backtest = perform_backtest(btc_csv_path)

for value in btc_backtest:
    print(value)

data: {
  "jupyter_id": "vraj2811",
  "result_type": "Main",
  "message": "Backtest completed",
  "result": {
    "static_statistics": {
      "From": "2019-09-08 17:57:00",
      "Total Trades": 30679,
      "Leverage Applied": 1.0,
      "Winning Trades": 4652,
      "Losing Trades": 26027,
      "No. of Long Trades": 15338,
      "No. of Short Trades": 15341,
      "Benchmark Return(%)": 350.354,
      "Benchmark Return(on $1000)": 3503.54,
      "Win Rate": 15.163467,
      "Winning Streak": 4,
      "Losing Streak": 64,
      "Gross Profit": -1996.376342,
      "Net Profit": -48014.876342,
      "Average Profit": -1.565073,
      "Maximum Drawdown(%)": 4757.613643,
      "Average Drawdown(%)": 2436.438169,
      "Largest Win": 99.201107,
      "Average Win": 5.310787,
      "Largest Loss": -114.239878,
      "Average Loss": -2.794047,
      "Maximum Holding Time": "0 days 12:14:59",
      "Average Holding Time": "0 days 0:25:25",
      "Maximum Adverse Excursion": 13.690258,
     